<a href="https://colab.research.google.com/github/DrJHSIM/PyTorch_Practice/blob/main/PyTorchQuickStartPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [1]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# dataload

In [5]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

testing_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [6]:
batch = 64
train_dataloader = DataLoader(training_data, batch_size = batch)
test_dataloader = DataLoader(testing_data, batch_size = batch)

# model

In [11]:
for X, y in train_dataloader:
    print(X.shape, y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [13]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [55]:
class SimpleNetworks(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.features = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.features(x)
        return x

model = SimpleNetworks().to(device)
print(model)

SimpleNetworks(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (features): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# train and valid define

In [56]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [57]:
def train(dataloader, loss_fn, optimizer, batch):
    model.train()
    for i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        preds = model(X)
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            print(f" Loss Value : {loss.item():>5f}, Current : [{(i+1)*batch}/{len(dataloader.dataset)}] ")

In [61]:
def test(dataloader, loss_fn):
    model.eval()
    test_loss = 0 # batch 사이즈의 평균 손실함수 값
    correction = 0  # 전체 맞은 갯수
    with torch.no_grad():
        for i, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss = loss_fn(preds, y).item()
            test_loss += loss
            correct = (preds.argmax(1) == y).type(torch.float).sum().item()
            correction += correct
        print(f"Test Loss Value : {test_loss/len(dataloader):>4f}, Acurracy : {correction/len(dataloader.dataset)*100:>0.1f}%")

In [62]:
epochs = 5
for i in range(epochs):
    print(f"{i+1} epoch \n -------------------------")
    train(train_dataloader, loss_fn, optimizer, batch)
    test(test_dataloader, loss_fn)
print("learning_done")

1 epoch 
 -------------------------
 Loss Value : 2.020241, Current : [64/60000] 
 Loss Value : 1.999288, Current : [6464/60000] 
 Loss Value : 1.896790, Current : [12864/60000] 
 Loss Value : 1.940438, Current : [19264/60000] 
 Loss Value : 1.843226, Current : [25664/60000] 
 Loss Value : 1.775932, Current : [32064/60000] 
 Loss Value : 1.802980, Current : [38464/60000] 
 Loss Value : 1.699887, Current : [44864/60000] 
 Loss Value : 1.723675, Current : [51264/60000] 
 Loss Value : 1.623551, Current : [57664/60000] 
Test Loss Value : 1.634991, Acurracy : 60.2%
2 epoch 
 -------------------------
 Loss Value : 1.686639, Current : [64/60000] 
 Loss Value : 1.657915, Current : [6464/60000] 
 Loss Value : 1.510994, Current : [12864/60000] 
 Loss Value : 1.584828, Current : [19264/60000] 
 Loss Value : 1.459040, Current : [25664/60000] 
 Loss Value : 1.435468, Current : [32064/60000] 
 Loss Value : 1.453610, Current : [38464/60000] 
 Loss Value : 1.370219, Current : [44864/60000] 
 Loss Val

# save and load

In [63]:
torch.save(model.state_dict(), "simplemodel.pth")
print(f"model save done to simplemodel.pth")

model save done to simplemodel.pth


In [64]:
model = SimpleNetworks().to(device) # 모델 불러오기
model.load_state_dict(torch.load("simplemodel.pth"))

<All keys matched successfully>

# inference

In [72]:
model.eval()
with torch.no_grad():
    for i, (X, y) in enumerate(test_dataloader):
        X, y = X.to(device), y.to(device)
        preds = model(X)
        correct = (preds.argmax(1) == y).type(torch.int)
        print(correct, f"\nbatch size correct : {correct.sum().item()/len(y)*100}%")
        print(f"(note : 1 means correct)")
        break

tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=torch.int32) 
batch size correct : 65.625%
(note : 1 means correct)
